In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.keras.layers import Dense
from numpy.random import default_rng
from sklearn.model_selection import train_test_split

In [2]:
# Loading pipeline params.
DATA_PATH = Path('data')
FILENAME_STOP = 'stop_90_diff_euler.csv'
FILENAME_0 = '0_90_diff_euler.csv'
FILENAME_90 = '90_90_diff_euler.csv'
FILENAME_180 = '180_90_diff_euler.csv'
FILENAME_270 = '270_90_diff_euler.csv'

NUM_TIMESTEPS = 6
IGNORE_FRACTION = 0.1
TEST_FRACTION = 0.25

# Training params.
LEARNING_RATE = 1e-5
EPOCHS = 100
BATCH_SIZE = 512

In [3]:
def load_data(
    path_stop,
    path_0,
    path_90,
    path_180,
    path_270,
    num_timesteps,
    ignore_fraction,
    test_fraction,
):

    def read_and_prepare_data(
        path,
        class_,
        num_timesteps,
        ignore_fraction,
        test_fraction,
    ):

        def split_temporally(X, test_fraction):
            num_samples = X.shape[0]
            num_train = int(num_samples * (1.0 - test_fraction))
            X_train, X_test = X[:num_train], X[num_train:]
            return X_train, X_test

        def transform_into_sequences(X, num_timesteps):
            num_samples = X.shape[0]
            X_seq = []
            for k in range(num_samples - num_timesteps + 1):
                X_seq.append(X[k:(k + num_timesteps)])
            X_seq = np.array(X_seq)
            y_seq = class_ * np.ones((X_seq.shape[0], 1))
            return X_seq, y_seq

        X = pd.read_csv(path).values
        X = X[:, [0,1,2,7,8,9, 10,11,12,17,18,19]]  # Use only feet translations.

        # Ignore beginning and end.
        num_samples = X.shape[0]
        num_ignore = int(ignore_fraction * num_samples)
        X = X[num_ignore:-num_ignore]

        # Split X temporally.
        X_train, X_test = split_temporally(X, test_fraction)

        # Transform X into short sequences.
        X_train, y_train = transform_into_sequences(X_train, num_timesteps)
        X_test, y_test = transform_into_sequences(X_test, num_timesteps)

        return X_train, y_train, X_test, y_test

    def shuffle_data(X, y):
        idx = np.arange(len(X))
        default_rng().shuffle(idx)
        X_shuffled = X[idx, :]
        y_shuffled = y[idx]
        return X_shuffled, y_shuffled

    X_train_stop, y_train_stop, X_test_stop, y_test_stop = \
        read_and_prepare_data(
            path_stop,
            0.0,
            num_timesteps,
            ignore_fraction,
            test_fraction,
        )
    
    X_train_0, y_train_0, X_test_0, y_test_0 = \
        read_and_prepare_data(
            path_0,
            1.0,
            num_timesteps,
            ignore_fraction,
            test_fraction,
        )

    X_train_90, y_train_90, X_test_90, y_test_90 = \
        read_and_prepare_data(
            path_90,
            2.0,
            num_timesteps,
            ignore_fraction,
            test_fraction,
        )
    
    X_train_180, y_train_180, X_test_180, y_test_180 = \
        read_and_prepare_data(
            path_180,
            3.0,
            num_timesteps,
            ignore_fraction,
            test_fraction,
        )
    
    X_train_270, y_train_270, X_test_270, y_test_270 = \
        read_and_prepare_data(
            path_270,
            4.0,
            num_timesteps,
            ignore_fraction,
            test_fraction,
        )

    X_train = np.concatenate((X_train_stop, X_train_0, X_train_90, X_train_180, X_train_270), axis=0)
    y_train = np.concatenate((y_train_stop, y_train_0, y_train_90, y_train_180, y_train_270), axis=0)
    X_test = np.concatenate((X_test_stop, X_test_0, X_test_90, X_test_180, X_test_270), axis=0)
    y_test = np.concatenate((y_test_stop, y_test_0, y_test_90, y_test_180, y_test_270), axis=0)

    X_train, y_train = shuffle_data(X_train, y_train)
    X_test, y_test = shuffle_data(X_test, y_test)

    return X_train, X_test, y_train, y_test


path_stop = Path(DATA_PATH, FILENAME_STOP)
path_0 = Path(DATA_PATH, FILENAME_0)
path_90 = Path(DATA_PATH, FILENAME_90)
path_180 = Path(DATA_PATH, FILENAME_180)
path_270 = Path(DATA_PATH, FILENAME_270)

X_train, X_test, y_train, y_test = load_data(
    path_stop,
    path_0,
    path_90,
    path_180,
    path_270,
    NUM_TIMESTEPS,
    IGNORE_FRACTION,
    TEST_FRACTION,
)

In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((100765, 6, 12), (100765, 1), (33573, 6, 12), (33573, 1))

In [5]:
X_test[60].shape


(6, 12)

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

CNN = Sequential(name="Sequential_CNN")

CNN.add(Conv2D(64, kernel_size=(4, 8),
               strides=(2, 2), padding="same",
               activation="selu", input_shape=[6, 12, 1]))

CNN.add(MaxPooling2D(pool_size=(2, 4), strides=(1, 2),
                     padding="valid"))

# Add another pair of Conv2D and MaxPooling2D for more model depth,
# followed by the flatten and multiple dense layers

CNN.add(Conv2D(32, kernel_size=(3, 3),
               strides=(1, 1), padding="same",
               activation="selu"))

CNN.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),
                     padding="valid"))

CNN.add(Flatten())

CNN.add(Dense(64, activation='selu'))
CNN.add(Dense(32, activation='selu'))
CNN.add(Dense(5, activation='softmax'))

CNN.summary()


# from functools import partial

# _, num_time, num_feat = X_train.shape

# activation = 'selu'

# DefaultConv1D = partial(
#     keras.layers.Conv1D,
#     kernel_size=1,
#     activation=activation,
#     padding="valid",
# )

# model = keras.models.Sequential([
#     DefaultConv1D(4 * num_feat, kernel_size=4, input_shape=[num_time, num_feat]),
#     keras.layers.BatchNormalization(),
#     DefaultConv1D(num_feat, kernel_size=2),
#     keras.layers.Flatten(),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(num_time * num_feat, activation=activation),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(num_feat, activation=activation),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(5, activation='softmax'),
# ])

Model: "Sequential_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_53 (Conv2D)          (None, 3, 6, 64)          2112      
                                                                 
 max_pooling2d_53 (MaxPoolin  (None, 2, 2, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 2, 2, 32)          18464     
                                                                 
 max_pooling2d_54 (MaxPoolin  (None, 1, 1, 32)         0         
 g2D)                                                            
                                                                 
 flatten_15 (Flatten)        (None, 32)                0         
                                                                 
 dense_45 (Dense)            (None, 64)             

In [73]:
from numpy import *
where_are_NaNsx = isnan(X_train)
X_train[where_are_NaNsx] = 0

where_are_NaNsy = isnan(y_train)
y_train[where_are_NaNsy] = 0

where_are_NaNsxt = isnan(X_test)
X_test[where_are_NaNsxt] = 0

where_are_NaNsyt = isnan(y_test)
y_test[where_are_NaNsyt] = 0

In [74]:
optimizer = keras.optimizers.Adam(learning_rate=1e-2)

CNN.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'],
)
history = CNN.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=256,
    validation_data=(X_test, y_test),
    verbose=1,
)

Epoch 1/100
394/394 [==============================] - 5s 11ms/step - loss: 0.8554 - accuracy: 0.6562 - val_loss: 2.6355 - val_accuracy: 0.4184
Epoch 2/100
394/394 [==============================] - 4s 11ms/step - loss: 0.4975 - accuracy: 0.6948 - val_loss: 3.1648 - val_accuracy: 0.4730
Epoch 3/100
394/394 [==============================] - 4s 11ms/step - loss: 0.5489 - accuracy: 0.6904 - val_loss: 1.9927 - val_accuracy: 0.4241
Epoch 4/100
394/394 [==============================] - 5s 11ms/step - loss: 0.4923 - accuracy: 0.6910 - val_loss: 2.9833 - val_accuracy: 0.4281
Epoch 5/100
394/394 [==============================] - 5s 13ms/step - loss: 0.4768 - accuracy: 0.6998 - val_loss: 2.6228 - val_accuracy: 0.5637
Epoch 6/100
394/394 [==============================] - 6s 15ms/step - loss: 0.4746 - accuracy: 0.7023 - val_loss: 3.6722 - val_accuracy: 0.5667
Epoch 7/100
394/394 [==============================] - 6s 14ms/step - loss: 0.4722 - accuracy: 0.7036 - val_loss: 3.8367 - val_accuracy:

KeyboardInterrupt: 

In [79]:
from functools import partial


activation = 'selu'

DefaultConv1D = partial(
    keras.layers.Conv1D,
    kernel_size=1,
    activation=activation,
    padding="valid",
)

model = keras.models.Sequential([
    DefaultConv1D(4 * num_feat, kernel_size=4, input_shape=[num_time, num_feat]),
    keras.layers.BatchNormalization(),
    DefaultConv1D(num_feat, kernel_size=2),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(num_time * num_feat, activation=activation),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(num_feat, activation=activation),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(5, activation='softmax'),
])
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'],
)

history = model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    verbose=1,
)

Epoch 1/100
197/197 [==============================] - 3s 10ms/step - loss: 0.6285 - accuracy: 0.5930 - val_loss: 11.1360 - val_accuracy: 0.5516
Epoch 2/100
197/197 [==============================] - 2s 8ms/step - loss: 0.5865 - accuracy: 0.6162 - val_loss: 17.7744 - val_accuracy: 0.4238
Epoch 3/100
197/197 [==============================] - 2s 9ms/step - loss: 0.5716 - accuracy: 0.6386 - val_loss: 9.2769 - val_accuracy: 0.4249
Epoch 4/100
197/197 [==============================] - 2s 9ms/step - loss: 0.5548 - accuracy: 0.6495 - val_loss: 6.6627 - val_accuracy: 0.3976
Epoch 5/100
197/197 [==============================] - 2s 9ms/step - loss: 0.5456 - accuracy: 0.6599 - val_loss: 6.1818 - val_accuracy: 0.4848
Epoch 6/100
197/197 [==============================] - 2s 9ms/step - loss: 0.5424 - accuracy: 0.6607 - val_loss: 7.7417 - val_accuracy: 0.4294
Epoch 7/100
197/197 [==============================] - 2s 9ms/step - loss: 0.5285 - accuracy: 0.6708 - val_loss: 7.0784 - val_accuracy: 0.4

Epoch 58/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4871 - accuracy: 0.7005 - val_loss: 116.3986 - val_accuracy: 0.2335
Epoch 59/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4758 - accuracy: 0.7018 - val_loss: 19.8763 - val_accuracy: 0.4260
Epoch 60/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4641 - accuracy: 0.7056 - val_loss: 14.8988 - val_accuracy: 0.4289
Epoch 61/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4638 - accuracy: 0.7043 - val_loss: 14.2009 - val_accuracy: 0.4268
Epoch 62/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4609 - accuracy: 0.7076 - val_loss: 16.6586 - val_accuracy: 0.5698
Epoch 63/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4622 - accuracy: 0.7018 - val_loss: 16.6957 - val_accuracy: 0.4256
Epoch 64/100
197/197 [==============================] - 2s 8ms/step - loss: 0.4600 - accuracy: 0.7084 - val_loss: 20.1670 - val_a

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
plt.plot(history.history['loss'][10:])
plt.plot(history.history['val_loss'][10:])
plt.yscale('log')

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred= np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:
y_pred[0]

In [ ]:
df = pd.DataFrame({
    'y_test': y_test.ravel().astype(int).tolist(),
    'y_pred': y_pred.ravel().astype(int).tolist(),
})

In [ ]:
pd.crosstab(df['y_test'], df['y_pred'])

In [ ]:
model.save('model.h5')

In [ ]:
X_train[0]

In [ ]:
np.argmax(model.predict(np.array([[[0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08],
       [0.08,  0.08,  0.08,  0.08, -0.08, -0.08, -0.08,  0.08]]])))

In [ ]:
x_new = X_test[0,:,:]

In [ ]:
argmax(model.predict(np.array([[[-0.04,  0.69,  0.72,  0.07, -0.19, -0.63, -0.71,  0.23],
       [-0.13,  0.69,  0.7 ,  0.14, -0.19, -0.64, -0.71,  0.23],
       [-0.22,  0.67,  0.67,  0.23, -0.19, -0.64, -0.71,  0.23],
       [-0.25,  0.67,  0.65,  0.25, -0.19, -0.64, -0.71,  0.23],
       [-0.31,  0.65,  0.63,  0.3 , -0.19, -0.64, -0.71,  0.23],
       [-0.35,  0.63,  0.6 ,  0.34, -0.19, -0.64, -0.71,  0.23],
       [-0.34,  0.64,  0.61,  0.33, -0.19, -0.64, -0.71,  0.23],
       [-0.34,  0.64,  0.61,  0.33, -0.19, -0.64, -0.71,  0.23],
       [-0.24,  0.67,  0.66,  0.25, -0.19, -0.64, -0.71,  0.23],
       [-0.12,  0.69,  0.7 ,  0.14, -0.19, -0.64, -0.71,  0.23]]])))

In [ ]:
x_new.shape

In [ ]:
(1,) + x_new.shape

In [ ]:
X_new = x_new.reshape((1,) + x_new.shape)
fast_or_slow = saved_model(X_new)

In [ ]:
float(fast_or_slow[0, 0])

In [ ]:
print(X_new.tolist())


In [ ]:
CNN.save('model_90_86.h5')

In [ ]:
np.fromstring(X_new.tobytes(), dtype=float).reshape(1,10,4)

In [ ]:
comma_string = "[[[-1.402271e-03 -2.203955e-08 7.028995e-07 9.999990e-01 -2.328306e-10 -6.455648e-09 7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00]]])"

In [ ]:
spaces_string = comma_string.replace(' ', ',')
spaces_string

In [ ]:
import ast
input = np.array([[[0.08, 0.14, 0.09, 0.18],
       [0.08, 0.14, 0.09, 0.22],
       [0.08, 0.14, 0.08, 0.17],
       [0.08, 0.14, 0.08, 0.1 ],
       [0.08, 0.14, 0.07, 0.08],
       [0.08, 0.14, 0.07, 0.08],
       [0.09, 0.2 , 0.07, 0.08],
       [0.1 , 0.32, 0.07, 0.07],
       [0.1 , 0.29, 0.07, 0.07],
       [0.09, 0.24, 0.07, 0.07]]])
    
lista_string="[0.08, 0.14, 0.09, 0.18]"
lista= ast.literal_eval(lista_string)
input = np.insert(input[0], 0,lista).reshape(1,11,4)
input = input[:,:-1,:]
input

In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
/content/my_model

In [ ]:
!zip -r model.zip model/ 

In [ ]:
print(json_str)

In [ ]:
input = np.array([[[ 2.66, -1.94,  0.66, -0.07],
       [ 2.66, -1.94,  0.66, -0.07],
       [ 2.65, -2.1 ,  0.66, -0.07],
       [ 2.65, -2.2 ,  0.66, -0.07],
       [ 2.65, -2.22,  0.66, -0.07],
       [ 2.65, -2.22,  0.68,  0.01],
       [ 2.64, -2.22,  0.68,  0.18],
       [ 2.64, -2.22,  0.69,  0.26],
       [ 2.64, -2.22,  0.67,  0.16],
       [ 2.64, -2.22,  0.67,  0.03]]])

In [ ]:
import ast
lista_string="[ 2.66, -1.94,  0.66, -0.07]"
lista= ast.literal_eval(lista_string)
input = np.append(input[0],[lista],axis=0).reshape(1,11,4)
input = input[:,1:,:]

In [ ]:
input

In [ ]:
X_train[0]

In [ ]:
import re

s = "[[[-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00] [-1.402271e-03 -2.203955e-08  7.028995e-07  9.999990e-01 -2.328306e-10 -6.455648e-09  7.037297e-07  1.000000e+00]]]"

def add_comma(match):
    return match.group(0) + ','

s = re.sub(r'\[[0-9\.\s]+\]', add_comma, s)
s = re.sub(r'([0-9\.]+)', add_comma, s)
mylist = eval(s)

In [ ]:
mylist